In [1]:
import sys
sys.path.append('../')

In [3]:
import itertools
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from pathlib import Path

from models.train import test
from models.utils import ContagionDataset, set_seed, ConfusionMatrix
from sklearn.metrics import matthews_corrcoef, mean_squared_error
from models.results import ResultCollection

In [74]:
seed = 4444

metric_filter_1 = 'val_mcc'
metric_filter_2 = 'test_mcc'

data_dir_name = 'data'
save_path_name = 'saved_'
networks = ['sym_network', 'europe_network']

target = 'additional_stress'
dict_sets_lengths = {
    '75':(0.5, 0.25, 0.25),
    '40':(0.3, 0.1, 0.6),
    '10':(0.07, 0.03, 0.9),
}


In [75]:
col = ResultCollection()

Get all models that will be tested

In [76]:
paths = itertools.chain.from_iterable([[k for k in Path(n).glob(f"models*/{save_path_name}*") if k.is_dir()] for n in networks])

In [78]:
for p in paths:
    name = p.name.split(save_path_name)[1]
    sets_lengths = dict_sets_lengths[p.parent.name.split('_')[2]]
    data_dir = p.parent.parent.joinpath(data_dir_name)
    
    set_seed(seed)
    dataset_val = ContagionDataset(
        raw_dir=data_dir,
        drop_edges=0,
        sets_lengths=sets_lengths,
        target = target,
        add_self_loop=True,
    )

    r = test(
        dataset=dataset_val,
        save_path=str(p),
        n_runs=1,
        debug_mode=False,
        use_cpu=False,
        save=True,
        use_edge_weight=True,
    )

    result = col.add(r)

    break

sym_network\models_clas_10\saved_gat
